## Scenario 3: Multiple data scientists working on multiple ML models

MLflow setup:
* Tracking server: yes, remote server (EC2).
* Backend store: postgresql database.
* Artifacts store: s3 bucket.

The experiments can be explored by accessing the remote server.

The example uses AWS to host a remote server. In order to run the example you'll need an AWS account. Follow the steps described in the file `mlflow_on_aws.md` to create a new AWS account and launch the tracking server. 

In [5]:
import mlflow
import os

os.environ["AWS_PROFILE"] = "manjakasoa" # fill in with your AWS profile. More info: https://docs.aws.amazon.com/sdk-for-java/latest/developer-guide/setup.html#setup-credentials

TRACKING_SERVER_HOST = "ec2-51-20-140-166.eu-north-1.compute.amazonaws.com" # fill in with the public DNS of the EC2 instance
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

In [6]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://ec2-51-20-140-166.eu-north-1.compute.amazonaws.com:5000'


In [7]:
mlflow.search_experiments() # list_experiments API has been removed, you can use search_experiments instead.()

[<Experiment: artifact_location='s3://mlflow-artifacts-remote-manza/1', creation_time=1754401296725, experiment_id='1', last_update_time=1754401296725, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='s3://mlflow-artifacts-remote-manza/0', creation_time=1754400696299, experiment_id='0', last_update_time=1754400696299, lifecycle_stage='active', name='Default', tags={}>]

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2025/08/05 14:17:12 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/05 14:17:15 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI: 's3://mlflow-artifacts-remote-manza/1/84762f51692b4e3294b69a3c86149c40/artifacts'
🏃 View run burly-pug-547 at: http://ec2-51-20-140-166.eu-north-1.compute.amazonaws.com:5000/#/experiments/1/runs/84762f51692b4e3294b69a3c86149c40
🧪 View experiment at: http://ec2-51-20-140-166.eu-north-1.compute.amazonaws.com:5000/#/experiments/1


In [9]:
mlflow.search_experiments()

[<Experiment: artifact_location='s3://mlflow-artifacts-remote-manza/1', creation_time=1754401296725, experiment_id='1', last_update_time=1754401296725, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='s3://mlflow-artifacts-remote-manza/0', creation_time=1754400696299, experiment_id='0', last_update_time=1754400696299, lifecycle_stage='active', name='Default', tags={}>]

### Interacting with the model registry

In [10]:
from mlflow.tracking import MlflowClient


client = MlflowClient(f"http://{TRACKING_SERVER_HOST}:5000")

In [11]:
client.search_registered_models()

[]

In [12]:
run_id = client.search_runs(experiment_ids=['1'])[0].info.run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2025/08/05 14:26:51 WARNING mlflow.tracking._model_registry.fluent: Run with id 84762f51692b4e3294b69a3c86149c40 has no artifacts at artifact path 'models', registering model based on models:/m-15dc23a5f2ef40408c886391f24d679f instead
2025/08/05 14:26:52 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1754404012124, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1754404012124, metrics=None, model_id=None, name='iris-classifier', params=None, run_id='84762f51692b4e3294b69a3c86149c40', run_link='', source='models:/m-15dc23a5f2ef40408c886391f24d679f', status='READY', status_message=None, tags={}, user_id='', version='1'>